In [29]:
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Toy dataset

In [30]:
data = torch.tensor([[1, 0, 1, 0],
                     [0, 0, 1, 0],
                     [1, 1, 1, 1],
                     [1, 1, 1, 0],
                     [1, 0, 0, 0],
                     [0, 0, 0, 0],
                     [1, 1, 0, 1],
                     [0, 1, 0, 0]
                     ], dtype=torch.float)
labels = torch.tensor([1, 0, 1, 1, 0, 0, 1, 0]).T

labels

tensor([1, 0, 1, 1, 0, 0, 1, 0])

# Fully Connected Network

In [38]:
class DenseNetwork(nn.Module):
    def __init__(self):
        super(DenseNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.Dropout(p=0.5),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
            nn.Sigmoid(),
            )

    def forward(self, x):
        x = self.net(x)
        return x

In [46]:
model = DenseNetwork()
if torch.cuda.is_available():
    model = model.cuda()
    print('cuda available')
print(model)

model.eval()

if torch.cuda.is_available():
    print(model(data.cuda()))
else:
    print(model(data))

cuda available
DenseNetwork(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): ReLU()
    (5): Linear(in_features=4, out_features=2, bias=True)
    (6): ReLU()
    (7): Linear(in_features=2, out_features=1, bias=True)
    (8): Sigmoid()
  )
)
tensor([[0.5650],
        [0.5650],
        [0.5650],
        [0.5650],
        [0.5650],
        [0.5650],
        [0.5650],
        [0.5650]], device='cuda:0', grad_fn=<SigmoidBackward>)


## Optimizer and Loss Criterion

In [56]:
# create a stochastic gradient descent optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
# create a loss function - binary cross-entropy
criterion = nn.BCELoss()

## Data Loader

In [48]:
params = {'batch_size': 2,
          'shuffle': True,
          'num_workers': 2,
          'pin_memory' : True}

train_loader = DataLoader(TensorDataset(data, labels), **params)
for batch_idx, batch in enumerate(train_loader):
    print(batch_idx, batch)

0 [tensor([[1., 1., 0., 1.],
        [1., 0., 1., 0.]]), tensor([1, 1])]
1 [tensor([[0., 0., 0., 0.],
        [1., 1., 1., 0.]]), tensor([0, 1])]
2 [tensor([[0., 1., 0., 0.],
        [0., 0., 1., 0.]]), tensor([0, 0])]
3 [tensor([[1., 0., 0., 0.],
        [1., 1., 1., 1.]]), tensor([0, 1])]


# Training

In [55]:
log_interval = 100
epochs = 200
model.train()

# training loop
for epoch in range(epochs):
    for batch_idx, (batch, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            batch, target = batch.cuda(), target.cuda()
        # resize data
        #data = data.view(-1, 4)
        optimizer.zero_grad()
        output = model(batch)
        loss = criterion(output, target.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.data))

Train Epoch: 0 [0/8 (0%)]	Loss: 0.811884
Train Epoch: 1 [0/8 (0%)]	Loss: 0.703495
Train Epoch: 2 [0/8 (0%)]	Loss: 0.615301
Train Epoch: 3 [0/8 (0%)]	Loss: 0.698886
Train Epoch: 4 [0/8 (0%)]	Loss: 0.693518
Train Epoch: 5 [0/8 (0%)]	Loss: 0.595297
Train Epoch: 6 [0/8 (0%)]	Loss: 0.859790
Train Epoch: 7 [0/8 (0%)]	Loss: 0.693300
Train Epoch: 8 [0/8 (0%)]	Loss: 0.693971
Train Epoch: 9 [0/8 (0%)]	Loss: 0.695116
Train Epoch: 10 [0/8 (0%)]	Loss: 0.693301
Train Epoch: 11 [0/8 (0%)]	Loss: 0.696422
Train Epoch: 12 [0/8 (0%)]	Loss: 0.680013
Train Epoch: 13 [0/8 (0%)]	Loss: 0.654817
Train Epoch: 14 [0/8 (0%)]	Loss: 0.693729
Train Epoch: 15 [0/8 (0%)]	Loss: 0.693979
Train Epoch: 16 [0/8 (0%)]	Loss: 0.801673
Train Epoch: 17 [0/8 (0%)]	Loss: 0.693208
Train Epoch: 18 [0/8 (0%)]	Loss: 0.690119
Train Epoch: 19 [0/8 (0%)]	Loss: 0.694779
Train Epoch: 20 [0/8 (0%)]	Loss: 0.693992
Train Epoch: 21 [0/8 (0%)]	Loss: 0.693340
Train Epoch: 22 [0/8 (0%)]	Loss: 0.779768
Train Epoch: 23 [0/8 (0%)]	Loss: 0.686201
Tr

# Predict

In [45]:
model.eval()

with torch.no_grad():
    if torch.cuda.is_available():
        output = model(data.cuda())
    else:
        output = model(data.cuda())

output

tensor([[7.8667e-01],
        [6.7580e-04],
        [7.8667e-01],
        [7.8667e-01],
        [2.9068e-04],
        [1.6755e-07],
        [7.8667e-01],
        [1.5554e-05]], device='cuda:0')